In [1]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [2]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00


In [3]:
#ini tidak dipakai untuk capstone, hanya untuk keperluan testing
#digunakan untuk membentuk frames dari sebuah video

import cv2
import os

def video_to_frames(video_path, output_folder):
    # Membuat folder output jika belum ada
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Membuka video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Inisialisasi counter frame
    frame_count = 0

    while True:
        # Membaca frame dari video
        ret, frame = cap.read()

        if not ret:
            break

        # Menyimpan frame ke folder output
        frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_filename, frame)

        frame_count += 1

    # Release video capture object
    cap.release()
    print(f"Total frames extracted: {frame_count}")

# Contoh penggunaan fungsi
video_path = '/content/WIN_20240603_15_00_18_Pro.mp4'  # Path ke file video Anda
output_folder = '/content/frames'  # Path ke folder output Anda

video_to_frames(video_path, output_folder)

Total frames extracted: 425


In [18]:
# ini yang terpakai untuk capstone
# deteksi objek dengan input frame, hasil processing disimpan ke folder baru

import os
import cv2
import torch
import matplotlib.pyplot as plt

def detect_objects_from_folder(model_path, folder_path, object_labels, fps):
    # Load frames from the folder in one line
    frames = [cv2.imread(os.path.join(folder_path, filename)) for filename in sorted(os.listdir(folder_path)) if filename.endswith(('.png', '.jpg', '.jpeg'))]

    # Load YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)

    # Variables to track detected object and its duration
    detected_object = None
    start_time = None
    total_duration = {label: 0 for label in object_labels}

    processed_frames = []

    # Process each frame
    for frame_count, frame in enumerate(frames):
        # Perform object detection on the frame
        results = model(frame)

        # Flags to indicate if any object is detected in the current frame
        object_detected = {label: False for label in object_labels}

        # Draw bounding boxes on the frame
        for detection in results.xyxy[0]:
            x1, y1, x2, y2, confidence, class_idx = detection
            label = model.names[int(class_idx)]

            # Check if the label is in the object_labels list
            if label in object_labels:
                # Convert coordinates to integers
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Add label and confidence score
                text = f'{label}: {confidence:.2f}'
                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Track detected objects
                object_detected[label] = True

        # Update detected objects and their durations
        for label in object_labels:
            if object_detected[label]:
                if detected_object != label:
                    detected_object = label
                    start_time = frame_count  # Use frame count instead of timestamp for simplicity
            else:
                if detected_object == label:
                    end_time = frame_count
                    duration = end_time - start_time
                    total_duration[label] += duration
                    print(f'{label} detected for {duration} frames')
                    detected_object = None

        # Append processed frame to list
        processed_frames.append(frame)

    # Convert frame duration to seconds
    total_duration_seconds = {label: duration / fps for label, duration in total_duration.items()}

    # Print total duration of the detected objects in seconds
    for label, duration in total_duration_seconds.items():
        print(f'Total duration of {label}: {duration:.2f} seconds')

    # Save processed frames to folder
    processed_frames_dir = '/content/processed_framesfixbgtplis'
    if not os.path.exists(processed_frames_dir):
        os.makedirs(processed_frames_dir)

    for i, frame in enumerate(processed_frames):
        cv2.imwrite(f'{processed_frames_dir}/frame_{i:04d}.jpg', frame)

    # Release the OpenCV window
    cv2.destroyAllWindows()

    return processed_frames, total_duration_seconds


# Example usage
model_path = '/content/weightsbest (1).pt'  # Path to YOLOv5 model
folder_path = '/content/frames'  # Path to folder containing frames
object_labels = ['bottle', 'cigarette', 'phone', 'smoke', 'vape']  # Object labels to detect
fps = 30  # Frame rate of the original video

processed_frames, durations = detect_objects_from_folder(model_path, folder_path, object_labels, fps)

print('Durations for each object:')
for label, duration in durations.items():
    print(f'{label}: {duration:.2f} seconds')




Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-6-9 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


phone detected for 20 frames
phone detected for 1 frames
phone detected for 1 frames
bottle detected for 1 frames
bottle detected for 4 frames
bottle detected for 1 frames
bottle detected for 10 frames
bottle detected for 3 frames
cigarette detected for 1 frames
bottle detected for 13 frames
bottle detected for 1 frames
bottle detected for 2 frames
bottle detected for 1 frames
cigarette detected for 1 frames
cigarette detected for 1 frames
bottle detected for 20 frames
bottle detected for 2 frames
bottle detected for 1 frames
bottle detected for 2 frames
bottle detected for 4 frames
cigarette detected for 1 frames
bottle detected for 3 frames
bottle detected for 1 frames
Total duration of bottle: 2.30 seconds
Total duration of cigarette: 0.13 seconds
Total duration of phone: 0.73 seconds
Total duration of smoke: 0.00 seconds
Total duration of vape: 0.00 seconds
Durations for each object:
bottle: 2.30 seconds
cigarette: 0.13 seconds
phone: 0.73 seconds
smoke: 0.00 seconds
vape: 0.00 sec

In [19]:
# INI DIPAKAI UNTUK MENJADIKAN PROCESSED_FRAMES DIJADIIN 1 VIDEO

# nama file output video dan pathnya
output_video_path = '/content/output_video.mp4'

# ukuran dari salah satu frame
height, width, _ = processed_frames[0].shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Format codec untuk file MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# setiap frame yang telah diproses "ditulis" ke dalam file video
for frame in processed_frames:
    out.write(frame)

out.release()

print(f'Video berhasil disimpan di: {output_video_path}')


Video berhasil disimpan di: /content/output_video.mp4


In [ ]:
#INI INPUTNYA VIDEO LANGSUNG

import torch
import cv2

def detect_objects_in_video(model_path, video_path, output_video_path, object_labels):
    # Load YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)

    # Open video file
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define video writer
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    # Variables to track detected object and its duration
    detected_object = None
    start_time = None
    total_duration = {label: 0 for label in object_labels}

    # Process each frame of the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object detection on the frame
        results = model(frame)

        # Flags to indicate if any object is detected in the current frame
        object_detected = {label: False for label in object_labels}

        # Draw bounding boxes on the frame
        for detection in results.xyxy[0]:
            x1, y1, x2, y2, confidence, class_idx = detection
            label = model.names[int(class_idx)]

            # Convert coordinates to integers
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Add label and confidence score
            text = f'{label}: {confidence:.2f}'
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Track detected objects
            if label in object_labels:
                object_detected[label] = True

        # Update detected objects and their durations
        for label in object_labels:
            if object_detected[label]:
                if detected_object != label:
                    detected_object = label
                    start_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert milliseconds to seconds
            else:
                if detected_object == label:
                    end_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert milliseconds to seconds
                    duration = end_time - start_time
                    total_duration[label] += duration
                    print(f'{label} detected for {duration:.2f} seconds')
                    detected_object = None

        # Write output frame to video writer
        out.write(frame)

    # Release video capture and writer
    cap.release()
    out.release()

    # Print total duration of the detected objects
    for label, duration in total_duration.items():
        print(f'Total duration of {label}: {duration:.2f} seconds')

    return output_video_path, total_duration

# Contoh penggunaan fungsi
model_path = '/content/weightsbest (1).pt'
video_path = '/content/WIN_20240603_15_00_18_Pro.mp4'
output_video_path = 'output_video.mp4'
object_labels = ['bottle', 'cigarette', 'phone', 'smoke', 'vape']

output_video, durations = detect_objects_in_video(model_path, video_path, output_video_path, object_labels)

print(f'Output video saved to {output_video}')
print('Durations for each object:')
for label, duration in durations.items():
    print(f'{label}: {duration:.2f} seconds')


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-6-8 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


phone detected for 0.67 seconds
phone detected for 0.03 seconds
phone detected for 0.03 seconds
bottle detected for 0.03 seconds
bottle detected for 0.13 seconds
bottle detected for 0.03 seconds
bottle detected for 0.34 seconds
bottle detected for 0.11 seconds
cigarette detected for 0.03 seconds
bottle detected for 0.43 seconds
bottle detected for 0.06 seconds
bottle detected for 0.03 seconds
cigarette detected for 0.03 seconds
cigarette detected for 0.03 seconds
bottle detected for 0.67 seconds
bottle detected for 0.06 seconds
bottle detected for 0.03 seconds
bottle detected for 0.06 seconds
bottle detected for 0.13 seconds
cigarette detected for 0.03 seconds
bottle detected for 0.03 seconds
bottle detected for 0.18 seconds
Total duration of bottle: 2.34 seconds
Total duration of cigarette: 0.13 seconds
Total duration of phone: 0.74 seconds
Total duration of smoke: 0.00 seconds
Total duration of vape: 0.00 seconds
Output video saved to output_video.mp4
Durations for each object:
bottl